In [ ]:
import numpy as np
import os

data_path = os.getenv("HOME")+'/aiffel/speech_recognition/data/speech_wav_8000.npz'
speech_data = np.load(data_path)

In [ ]:
print("Wave data shape : ", speech_data["wav_vals"].shape)
print("Label data shape : ", speech_data["label_vals"].shape)

In [ ]:
speech_data1 = []
speech_data1.append(speech_data["wav_vals"][0:10000])
speech_data1 = np.array(speech_data1)

In [ ]:
import librosa

def wav2spec(wav, fft_size=258): # spectrogram shape을 맞추기위해서 size 변형
    D = np.abs(librosa.stft(wav, n_fft=fft_size))
    return D

In [ ]:
spec_data=[]
n = 0

while n != 10000:
    print(n)
    spec_data.append(wav2spec(speech_data["wav_vals"][n]))
    n += 1
    
spec_data = np.array(spec_data)
print(spec_data.shape)